# HPEP API Development


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, classification_report, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer

from src.eda import *
from src.config import *
from src.data_extract import *
from src import feature_selection as fs
from src.evaluation import *


# 1. Data Extraction


## 1.2. Extract data.

In [2]:
mhas = get_data_frame()
df = extract(mhas)

/home/giraldiego/Desktop/code/workspaces/anyoneai/final_project/victoria/src/data_extract.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_s1[p1] = df[s1].copy()
/home/giraldiego/Desktop/code/workspaces/anyoneai/final_project/victoria/src/data_extract.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_s1[p1] = df[s1].copy()
/home/giraldiego/Desktop/code/workspaces/anyoneai/final_project/victoria/src/data_extract.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

## 2.1. Identification of the target variable
* The target variable belongs to *Section C: Health Care Utilization and Insurance* and is labeled **Medical Care Utilization: Hospital** *`(r/s)#hosp1y`*, where *"r"* refers to "respondent" and *"s"* to "spouse".  Finally, *"#"* goes from 1 to 5 and refers to waves. 

* *rwhosp1y* indicates whether the respondent reports at least one overnight hospital stay in the last 12 months. RwHOSP1Y is coded as 0 if the respondent had no overnight hospital stays, and is coded as 1 if the respondent had at least one overnight hospital stay. 

* *swhosp1y* is taken from the spouse value in wave "w" for *rwhosp1y*. In addition to the special default codes used in *rwhosp1y*, if the respondent is not considered matched in the current cycle and is assumed to be single, a special default value of .u is used. In addition, if the respondent is not considered matched in the current cycle but reports being married, a special default value of .u is used. Similarly, if the respondent is not considered matched in the current cycle but reports being married, a special omitted value of .v is used.

In [3]:
# renaming the dataframe for convenience
target = "pahosp1y"

# 3. Model


## 3.1. Feature Selection and Preprocessing

In [4]:
# filtering the data by elderly people only.
df = df.loc[df['paagey']>=50]

# The selected features, we believe are relevant to the model.
selected=['pahosp1y','pahip50e', 'padadage', 'pamomage', 
          'pahibpe', 'parespe', 'pacholst', 'pagrossaa', 
          'paosleep', 'pafallinj', 'pacancre', 'paswell', 
          'parafaany', 'pawheeze', 'paagey', 
          'paarthre', 'padiabe', 'parjudg', 'paweight', 
          'paclims', 'pastroke', 'pasmokev', 
          'pahearaid', 'pameds', 'paheight', 'paglasses', 
          'paeat', 'papaina', 'pagender', 'padrinkb']

print(f"There are {len(selected)} features selected")

df3 = df.where(df['cpindex'] <=  109).dropna(subset=['cpindex'])

df_s = df3.dropna(subset=[target]) 
df_s = df_s[selected].copy()

nulls_percentage = df_s.isnull().sum(axis=1) / len(df.columns)
df_s = df_s[(df_s[target] != 0) | (nulls_percentage <= 0.5)]

# splitting the data into X and y
y_df = df_s[target]
X_df = df_s.drop(target,axis=1)

There are 30 features selected


In [5]:
## Save dataframe for persistent use

import pickle

output_file = "df_s-dataset.bin"

with open(output_file, "wb") as f_out:
    pickle.dump(df_s, f_out)

In [6]:
from sklearn.model_selection import train_test_split

# Split the dataset in train and test data
df_train, df_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42, shuffle=True)

# Imputing nan
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent') # Imputer using the mode
X_train = imputer.fit_transform(df_train) 

# Scaling the train data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Normalizing the train data
transformer = PowerTransformer(method='yeo-johnson')
X_train = transformer.fit_transform(X_train)

## Now we apply the same fot test dataset
X_test = imputer.transform(df_test)

# Scaling the test data
X_test = scaler.transform(X_test)

# Normalizing the test data
X_test = transformer.transform(X_test)

## 3.2. Logistic Regression Model

In [7]:
from sklearn.linear_model import LogisticRegression
import time

log_reg = None
log_reg = LogisticRegression(C=0.0001)
log_reg.fit(X_train, y_train)

# Train data predictions (class 1)
log_reg_train = log_reg.predict_proba(X_train)[:, 1]

# Validation data predictions (class 1)
log_reg_val = log_reg.predict_proba(X_test)[:, 1]

# Train ROC AUC Score
roc_auc_train = roc_auc_score(y_true=y_train, y_score=log_reg_train)
print(f"Train ROC AUC Score: {roc_auc_train:.4f}")

# Validation ROC AUC Score
roc_auc_val = roc_auc_score(y_true=y_test, y_score=log_reg_val)
print(f"Validation ROC AUC Score: {roc_auc_val:.4f}")

Train ROC AUC Score: 0.7365
Validation ROC AUC Score: 0.7395


## 3.3. Random Forest Model

In [8]:
from sklearn.ensemble import RandomForestClassifier
import time

# Initialize RandomForest classifier with n_jobs=-1 to use all CPU cores
rf = RandomForestClassifier(n_jobs=-1)

# Measure start time
start_time = time.time()

# Train the model
rf.fit(X_train, y_train)

# Measure completion time
end_time = time.time()

# Calculate elapsed time
training_time = end_time - start_time

# Print training time
print(f"Training time: {training_time} seconds")

# The trained model is in the variable 'rf'

# Train data predictions (class 1)
rf_pred_train = rf.predict_proba(X_train)[:, 1]

# Validation data predictions (class 1)
rf_pred_test = rf.predict_proba(X_test)[:, 1]


# Train ROC AUC Score
roc_auc_train = roc_auc_score(y_true=y_train, y_score=rf_pred_train)
print(f"Train ROC AUC Score: {roc_auc_train:.4f}")

# Validation ROC AUC Score
roc_auc_test = roc_auc_score(y_true=y_test, y_score=rf_pred_test)
print(f"Validation ROC AUC Score: {roc_auc_test:.4f}")

Training time: 3.3524460792541504 seconds
Train ROC AUC Score: 1.0000
Validation ROC AUC Score: 0.9387


# 5. Exporting the model

## 5.1 Saving the model

In [10]:
model = rf
model

RandomForestClassifier(n_jobs=-1)

In [11]:
# scaler, transformer
transformers = {"imputer":imputer, "scaler":scaler, "transformer":transformer}
transformers

{'imputer': SimpleImputer(strategy='most_frequent'),
 'scaler': StandardScaler(),
 'transformer': PowerTransformer()}

In [12]:
import pickle

model_type = "RFC_model" # This is the name for the filename of the model

output_file = f"hospitalization-{model_type}.bin"

with open(output_file, "wb") as f_out:
    pickle.dump((transformers, model), f_out)


## 5.2 Load the model (Testing)

In [9]:
import pickle

model_type = "RFC_model" # This is the name for the filename of the model

input_file = f"hospitalization-{model_type}.bin" # Use the same name than above

with open(input_file, "rb") as f_in:
    transformers_imported, model_imported = pickle.load(f_in)

transformers_imported, model_imported

({'imputer': SimpleImputer(strategy='most_frequent'),
  'scaler': StandardScaler(),
  'transformer': PowerTransformer()},
 RandomForestClassifier(n_jobs=-1))

In [10]:
input_file = "df_s-dataset.bin"
with open(input_file, "rb") as f_in:
    df_s_imported = pickle.load(f_in)
df_s_imported.head()

,pahosp1y,pahip50e,padadage,pamomage,pahibpe,parespe,pacholst,pagrossaa,paosleep,pafallinj,...,pastroke,pasmokev,pahearaid,pameds,paheight,paglasses,paeat,papaina,pagender,padrinkb
0,1.0,1.0,78.0,60.0,1.0,0.0,1.0,0.0,NaN,1.0,...,0.0,1.0,0.0,0.0,1.73,0.0,NaN,0.0,1.0,0.0
1,0.0,0.0,NaN,70.0,0.0,0.0,1.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,2.0,0.0
2,0.0,0.0,83.0,32.0,0.0,0.0,0.0,0.0,NaN,NaN,...,0.0,1.0,0.0,0.0,NaN,1.0,0.0,1.0,1.0,0.0
4,0.0,0.0,75.0,82.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,1.0,0.0,0.0,1.77,0.0,NaN,NaN,1.0,1.0
6,0.0,0.0,78.0,85.0,1.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,1.0,0.0


In [11]:
# Import transformer objects from pickle
imputer_imported = transformers_imported["imputer"]
scaler_imported = transformers_imported["scaler"]
transformer_imported = transformers_imported["transformer"]

In [12]:
def extract_patient(pos, df):    
    """
    Extracts the target variable and patient information from the dataframe.

    Parameters:
    pos (int): The position of the patient in the dataframe.
    df (pandas.DataFrame): The dataframe containing patient information.

    Returns:
    tuple: A tuple containing the target variable (y_patient) and a dictionary (patient_dict) 
        representing the patient information.
    """
    target = "pahosp1y"
    y_patient = df[target].iloc[pos]
    # print(y_patient) # Print it the patient was hospitalizated 

    # Convert patient to dict similar to format of the json in the api
    patient_dict = df.drop(target, axis=1).iloc[pos].to_dict()
    # print(patient_dict)
    return y_patient, patient_dict

In [13]:
def predict_patient(patient_dict, transformers, model):
    """
    Predicts the hospitalization probability for a given patient.

    Parameters:
    - patient_dict (dict): A dictionary containing the patient's information.
    - transformers (dict): A dictionary containing the transformers used for data preprocessing.
    - model: The trained machine learning model.

    Returns:
    - hospitalization (bool): True if the patient is predicted to be hospitalized, False otherwise.
    - y_patient_pred (float): The predicted probability of hospitalization for the patient.
    """
    
    # From patient_dict
    df_patient = pd.Series(patient_dict).to_frame().T

    imputer = transformers["imputer"]
    scaler = transformers["scaler"]
    transformer = transformers["transformer"]

    ## Now we apply the transformations to the single patient dataframe
    X_patient = imputer.transform(df_patient)

    # Scaling the patient data
    X_patient = scaler.transform(X_patient)

    # Normalizing the patient data
    X_patient = transformer.transform(X_patient)

    y_patient_pred = model.predict_proba(X_patient)[0, 1]
    hospitalization = y_patient_pred >= 0.5

    return hospitalization, y_patient_pred

### Test model with a sample patient

In [24]:
df_s_imported[df_s_imported[target] == 1].head(10)

,pahosp1y,pahip50e,padadage,pamomage,pahibpe,parespe,pacholst,pagrossaa,paosleep,pafallinj,...,pastroke,pasmokev,pahearaid,pameds,paheight,paglasses,paeat,papaina,pagender,padrinkb
0,1.0,1.0,78.0,60.0,1.0,0.0,1.0,0.0,NaN,1.0,...,0.0,1.0,0.0,0.0,1.73,0.0,NaN,0.0,1.0,0.0
18,1.0,0.0,70.0,NaN,1.0,0.0,0.0,0.0,NaN,1.0,...,1.0,1.0,0.0,0.0,NaN,0.0,0.0,1.0,2.0,0.0
75,1.0,1.0,77.0,70.0,0.0,0.0,1.0,0.0,NaN,NaN,...,0.0,1.0,0.0,0.0,1.60,0.0,0.0,1.0,1.0,0.0
76,1.0,0.0,83.0,72.0,0.0,0.0,1.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,1.30,0.0,0.0,0.0,2.0,0.0
78,1.0,1.0,NaN,80.0,1.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,2.0,0.0
107,1.0,1.0,62.0,101.0,0.0,0.0,NaN,NaN,2.0,1.0,...,1.0,1.0,0.0,NaN,1.52,0.0,0.0,NaN,1.0,0.0
111,1.0,0.0,70.0,65.0,1.0,0.0,1.0,0.0,NaN,0.0,...,0.0,1.0,0.0,0.0,1.65,0.0,0.0,NaN,1.0,0.0
114,1.0,0.0,51.0,50.0,0.0,0.0,1.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.67,1.0,NaN,0.0,1.0,0.0
145,1.0,0.0,64.0,88.0,1.0,0.0,1.0,0.0,NaN,0.0,...,0.0,1.0,0.0,0.0,1.60,1.0,0.0,NaN,1.0,0.0
162,1.0,1.0,NaN,75.0,0.0,0.0,1.0,0.0,NaN,1.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,1.0,0.0


In [25]:
idx = 162  
y_patient = df_s_imported.loc[idx].iloc[0:1][target].iloc[0]
print(y_patient) # Print it the patient was hospitalizated 

# Convert patient to dict similar to format of the json in the api
patient_dict = df_s_imported.loc[idx].iloc[0:1].drop(target, axis=1).to_dict(orient="records")[0]
print(patient_dict)

1.0
{'pahip50e': 1.0, 'padadage': nan, 'pamomage': 75.0, 'pahibpe': 0.0, 'parespe': 0.0, 'pacholst': 1.0, 'pagrossaa': 0.0, 'paosleep': nan, 'pafallinj': 1.0, 'pacancre': 0.0, 'paswell': 1.0, 'parafaany': nan, 'pawheeze': 0.0, 'paagey': 64.0, 'paarthre': 1.0, 'padiabe': 1.0, 'parjudg': nan, 'paweight': 65.0, 'paclims': 1.0, 'pastroke': 0.0, 'pasmokev': 0.0, 'pahearaid': 0.0, 'pameds': 0.0, 'paheight': nan, 'paglasses': 0.0, 'paeat': nan, 'papaina': 1.0, 'pagender': 1.0, 'padrinkb': 0.0}


In [16]:
y_patient, predict_patient(patient_dict, transformers_imported, model_imported)

(1.0, (True, 0.7))

### 5.2.1 Working with json format

In [17]:
def clean_patient_dict(patient_dict):
    patient_clean = patient_dict.copy()
    for key, value in patient_clean.items():
        if isinstance(value, np.ndarray):
            patient_clean[key] = value.tolist()  # Convert ndarray to list for JSON compatibility
        if isinstance(value, float) and np.isnan(value):
            patient_clean[key] = ""  # Replace np.nan with string ""

    return patient_clean

In [18]:
patient_clean = clean_patient_dict(patient_dict)

In [19]:
import json

json_filename = "rfc-single_patient"
with open(f"{json_filename}.json", "w") as fp:
    json.dump(patient_clean, fp) 

### Test using json

In [20]:
input_file = "rfc-single_patient.json"

with open(input_file, "rb") as f_in:
    patient_json = json.load(f_in)
print(patient_json)

{'pahip50e': 1.0, 'padadage': '', 'pamomage': 75.0, 'pahibpe': 0.0, 'parespe': 0.0, 'pacholst': 1.0, 'pagrossaa': 0.0, 'paosleep': '', 'pafallinj': 1.0, 'pacancre': 0.0, 'paswell': 1.0, 'parafaany': '', 'pawheeze': 0.0, 'paagey': 64.0, 'paarthre': 1.0, 'padiabe': 1.0, 'parjudg': '', 'paweight': 65.0, 'paclims': 1.0, 'pastroke': 0.0, 'pasmokev': 0.0, 'pahearaid': 0.0, 'pameds': 0.0, 'paheight': '', 'paglasses': 0.0, 'paeat': '', 'papaina': 1.0, 'pagender': 1.0, 'padrinkb': 0.0}


In [21]:
def revive_nan(data):
    """
    Recursively replaces empty strings with NaN values in a nested dictionary or list.

    Parameters:
    data (dict or list): The input data to be processed.

    Returns:
    dict or list: The processed data with empty strings replaced by NaN values.
    """
    if isinstance(data, dict):
        for key, value in data.items():
            data[key] = revive_nan(value)
    elif isinstance(data, list):
        for i, value in enumerate(data):
            data[i] = revive_nan(value)
    elif data == "":
        return np.nan
    else:
        return data

In [22]:
revive_nan(patient_json)
patient_json

{'pahip50e': 1.0,
 'padadage': nan,
 'pamomage': 75.0,
 'pahibpe': 0.0,
 'parespe': 0.0,
 'pacholst': 1.0,
 'pagrossaa': 0.0,
 'paosleep': nan,
 'pafallinj': 1.0,
 'pacancre': 0.0,
 'paswell': 1.0,
 'parafaany': nan,
 'pawheeze': 0.0,
 'paagey': 64.0,
 'paarthre': 1.0,
 'padiabe': 1.0,
 'parjudg': nan,
 'paweight': 65.0,
 'paclims': 1.0,
 'pastroke': 0.0,
 'pasmokev': 0.0,
 'pahearaid': 0.0,
 'pameds': 0.0,
 'paheight': nan,
 'paglasses': 0.0,
 'paeat': nan,
 'papaina': 1.0,
 'pagender': 1.0,
 'padrinkb': 0.0}

In [23]:
y_patient, predict_patient(patient_json, transformers_imported, model_imported)

(1.0, (True, 0.7))

### 5.2.2 Run a batch of predictions in tests patients 

In [25]:
df_test.head()

,pahip50e,padadage,pamomage,pahibpe,parespe,pacholst,pagrossaa,paosleep,pafallinj,pacancre,...,pastroke,pasmokev,pahearaid,pameds,paheight,paglasses,paeat,papaina,pagender,padrinkb
13593,NaN,69.0,75.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,...,0.0,1.0,0.0,0.0,1.62,1.0,0.0,0.0,2.0,0.0
2455,1.0,NaN,55.0,1.0,0.0,1.0,0.0,NaN,1.0,0.0,...,0.0,1.0,0.0,0.0,1.45,1.0,NaN,0.0,2.0,0.0
14629,NaN,85.0,83.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,1.45,0.0,NaN,NaN,2.0,0.0
11219,0.0,32.0,84.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,...,0.0,1.0,0.0,0.0,1.80,1.0,NaN,NaN,1.0,0.0
4188,0.0,76.0,65.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,1.0,0.0,0.0,1.60,0.0,NaN,NaN,1.0,0.0


In [22]:
df_test_full = pd.concat([y_test, df_test], axis=1)

In [23]:
# Visualize 10 patients who were hospitalized
df_test_full[df_test_full["pahosp1y"] == 1].head(10)

,pahosp1y,pahip50e,padadage,pamomage,pahibpe,parespe,pacholst,pagrossaa,paosleep,pafallinj,...,pastroke,pasmokev,pahearaid,pameds,paheight,paglasses,paeat,papaina,pagender,padrinkb
13593,1.0,NaN,69.0,75.0,1.0,0.0,1.0,0.0,NaN,0.0,...,0.0,1.0,0.0,0.0,1.62,1.0,0.0,0.0,2.0,0.0
6327,1.0,0.0,NaN,84.0,0.0,0.0,NaN,NaN,3.0,0.0,...,0.0,0.0,0.0,NaN,1.50,1.0,0.0,NaN,2.0,0.0
21384,1.0,0.0,75.0,61.0,1.0,0.0,1.0,1.0,NaN,0.0,...,0.0,1.0,0.0,0.0,1.68,1.0,1.0,1.0,1.0,0.0
3695,1.0,0.0,NaN,70.0,1.0,0.0,1.0,1.0,NaN,0.0,...,0.0,0.0,0.0,0.0,1.54,1.0,0.0,NaN,2.0,0.0
6733,1.0,0.0,80.0,95.0,1.0,0.0,1.0,1.0,NaN,0.0,...,0.0,0.0,0.0,1.0,1.54,1.0,0.0,1.0,2.0,0.0
6915,1.0,NaN,70.0,78.0,0.0,1.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,1.50,0.0,NaN,NaN,2.0,0.0
5106,1.0,0.0,80.0,63.0,0.0,0.0,1.0,0.0,NaN,NaN,...,0.0,0.0,1.0,0.0,1.75,0.0,NaN,NaN,1.0,0.0
404,1.0,NaN,92.0,72.0,1.0,1.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,1.0,NaN,0.0,2.0,0.0
9953,1.0,0.0,70.0,40.0,1.0,0.0,0.0,1.0,NaN,0.0,...,0.0,1.0,0.0,1.0,1.52,0.0,0.0,NaN,1.0,0.0
2007,1.0,1.0,75.0,75.0,0.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,NaN,1.0,NaN,1.0,2.0,0.0


In [24]:
matches = 0
total = 1000
for pos in range(0,total):
    y_patient, patient_dict = extract_patient(pos, df_test_full)
    hospitalization, y_patient_pred = predict_patient(patient_dict, transformers_imported, model_imported)
    if y_patient == (hospitalization if 1 else 0):
        matches += 1    
print(matches/total)

0.953
